In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
pd.options.display.max_columns = 999

## Setup Stuff (that should come from config files or database later)

In [2]:
CS_TRANSACTION_FEE = 0.0101
CHANNELS = [
    {'name':'AP Fusion', 'channel_transaction_fee':0.08, 'target_profit': 0.09},

    {'name':'PS Amazon', 'channel_transaction_fee':0.119},
    {'name':'PS Walmart', 'channel_transaction_fee':0.125},
    {'name':'PS Ebay', 'channel_transaction_fee':0.17}, ## Changed from 0.15 on Feb 24 2022
    
    {'name':'BS Amazon', 'channel_transaction_fee':0.12},
    {'name':'BS Walmart', 'channel_transaction_fee':0.12},
    {'name':'BS Ebay', 'channel_transaction_fee':0.12},
    
    {'name':'Mecka', 'channel_transaction_fee': 0.12}
]

WAREHOUSES = [
    # Fully integrated warehouses
    {'key':'C', 'name':'Brock', 'shipping':'free-ish'},
    {'key':'D', 'name':'Dorman Direct', 'shipping':'free-ish', 'target_profit': 0.6},
    {'key':'J', 'name':'PFG', 'shipping':'theirs'},
    {'key':'K', 'name':'Keystone', 'shipping':'theirs'},
    {'key':'N', 'name':'NPW', 'shipping':'ours'},
    {'key':'O', 'name':'Tonsa', 'shipping':'ours'},
    {'key':'P', 'name':'Parts Auth', 'shipping':'theirs'},
    {'key':'Y', 'name':'Motor State', 'shipping':'theirs'},
    # Manual/FTP warehouses
    {'key':'1', 'name':'Jante Wheel', 'shipping':'free'},
    {'key':'2', 'name':'OE Wheels', 'shipping':'theirs'},
    {'key':'6', 'name':'Burco Mirrors', 'shipping':'ours'},
    {'key':'8', 'name':'Race Sport Lighting', 'shipping':'ours', 'target_profit': 0.1},
    {'key':'9', 'name':'Sunbelt APG', 'shipping':'ours'},
    # Low-volume, or unused warehouses
    #{'key':'5', 'name':'KSI Trading'},
    #{'key':'7', 'name':'NTW'},
    #{'key':'H', 'name':'Hanson'},
    #{'key':'3', 'name':'Motor State'},
]

def get_warehouse_key(warehouse_name):
    for warehouse in WAREHOUSES:
        if warehouse['name'] == warehouse_name:
            return warehouse['key']
    return None


DEFAULT_TARGET_PROFIT = 0.05
DEFAULT_SHIP_MARKUP = 1 / 1.12
EXCLUDED_WAREHOUSES = ['A','5','7','H','3']
PUNCTUATION_WAREHOUSES = ['J','1','C', '9', '8', 'Y']

PARTS_AUTH_SHIPPING_MODEL = 'shipping-research/tree-model.pkl'

PRICE_FILE_COLUMNS = ['CS-SKU-NP', 'MinPrice', 'Shipping', 'Carrier', 'Service', 'Markup',
       'ShipMkup', 'ListMkup', 'PackQty', 'MinQty', 'MaxQty', 'Zip Code',
       'CatSKU', 'OP-Lowest(Y)', 'VND-Lowest(Y)', 'MinMkDown', 'MaxMkUp', 'Interval',
       'BundleSK', 'Duplicate']

PRICE_FILE_LOCATION = 'price-files'

CatSKU_CHANNELS = ['PS Ebay']

# Main

#### Load in data.

In [3]:
pw = pd.read_csv('inventory/pw-all.csv', low_memory=False, 
                 dtype={'MasterLC':'Int64', 'Zip Code': str})
# pw['MasterLC'] = pw['MasterLC'].astype('Int64')

# Temporarily remove all NPW.
##pw = pw[pw['WD'] != 'N']

warehouses = pw['WD'].unique()

## Top-level processing and filtering.

#### Sad updates.

In [4]:
# Correct the Line Code and CS-SKU-NP for Brock
pw.loc[pw['WD']=='C', 'CS-SKU-NP'] = pw.loc[pw['WD']=='C', 'CS-SKU']
pw.loc[(pw['MasterLC']==158) & (pw['LC']=='429'), 'MasterLC'] = 429

#### Preprocess price file.

In [5]:
map_prices = pd.read_csv('maps.csv')
map_prices = map_prices.sort_values('CS-SKU-NP').drop_duplicates(subset=['CS-SKU-NP'])
map_prices = map_prices.set_index('CS-SKU-NP')['MAP']

In [6]:
# Define separate column for managing costs vs MinPrice to avoid confusion.
pw['item_cost'] = pw['MinPrice']

# update Dorman costs (which are per-pack initally) to be per-unit
dorman_update_idx = (pw['WD']=='D') & (pw['PackQty'].notna())
pw.loc[dorman_update_idx, 'item_cost'] = pw.loc[dorman_update_idx, 'item_cost'] / pw.loc[dorman_update_idx, 'PackQty']

# Make a "backup" copy of cs-sku-np for CatSKU situations
pw['CS-SKU-NP-CatSKU'] = pw['CS-SKU-NP']

# Set CSSKUNP depending on if it's a punctuation warehouse
x = pw[pw['WD'].isin(PUNCTUATION_WAREHOUSES)].copy()
x['CS-SKU-NP'] = x['WD'] + x['MasterLC'].astype(str) + '|' + x['Part Number']
pw.loc[pw['WD'].isin(PUNCTUATION_WAREHOUSES), :] = x

x = pw[~pw['WD'].isin(PUNCTUATION_WAREHOUSES)].copy()
x['CS-SKU-NP'] = (x['WD'] + x['MasterLC'].astype(str) + '|' 
                  + x['Part Number'].map(lambda s: ''.join(filter(str.isalnum, s))))
pw.loc[~pw['WD'].isin(PUNCTUATION_WAREHOUSES), :] = x

maps = []
for i, row in pw.iterrows():
    csskunp = row['CS-SKU-NP']
    if csskunp in map_prices:
        maps.append(map_prices[csskunp])
    else:
        maps.append(1)
pw['MAP'] = maps

# Remove WeatherTech (just in case)
pw = pw[pw['MasterLC'] != 310]

# Remove First Stop Brakes Dorman Line
# df = df[~((df['WD']=='D') & df['Part Number'].isin(first_stop_brakes))]
# Nope, actually don't remove them, just set MinQty really high... at the end.

# Remove placeholder values for Weight/ShipWeight
pw.loc[(pw['Weight']==9999), 'Weight'] = np.nan
pw.loc[(pw['ShipWeight']==9999), 'ShipWeight'] = np.nan

#### Filter parts.

In [7]:
# Idea here is to filter out all the lil nasties that we don't want to include.
# This could differ by warehouse, or not.
# Things like, heavy parts, big or oddly shaped parts, 
# parts that are really expensive, or come in packs of many.
# ... See notes on original Jim conversation for what all you should be including here.

# Filter out excluded warehouses.
pw = pw[~pw['WD'].isin(EXCLUDED_WAREHOUSES)]
# Filter out nasty pack quantities. (allow these for Dorman, since we have clean data.)
pw['PackQty'] = pw['PackQty'].fillna(1) # assume PackQty of NA => PackQty=1
pw = pw[(pw['PackQty'] <= 10) | (pw['WD']=='D')]

#### Calculate shipping by warehouse.

In [ ]:
shipping_data = pd.read_csv('shipping_data.csv', low_memory=False, parse_dates=['Ship Date'])
shipping_data['warehouse_key'] = shipping_data['Warehouse'].map(get_warehouse_key)
shipping_data = shipping_data[shipping_data['Quantity'] > 0]
shipping_data['Ship Cost'] = shipping_data['Ship Cost'] / shipping_data['Quantity']
shipping_data = shipping_data[shipping_data['Ship Cost'] > 0.05][['CS-SKU','warehouse_key','Ship Cost','Ship Date']]

price_file_shipping = pd.read_csv('price-file-shipping.csv')
price_file_shipping = {row['CS-SKU-NP']: row['Shipping'] for _, row in price_file_shipping.iterrows()}

def get_historical_shipping_estimates(df, warehouse_key): # (cssku, warehouse_key):    
    warehouse_shipping_data = shipping_data[(shipping_data['warehouse_key']==warehouse_key)].copy()
    # create warehouse-level shipping model
    feature_cols = ['Weight', 'DimWeight', 'ShipWeight', 'Length', 'Width', 'Height']
    ship_weights = df[['MasterSKU']+feature_cols].merge(warehouse_shipping_data, how='inner',
                                                        left_on='MasterSKU', right_on='CS-SKU').copy().dropna()
    ship_weights = ship_weights[(ship_weights['ShipWeight'] > 0) & (ship_weights['ShipWeight'] < 1000)]
    if len(ship_weights) > 0:
        model = LinearRegression().fit(ship_weights[feature_cols], ship_weights['Ship Cost'])
        # log model error for audit purposes
        print('RMSE:',mean_squared_error(ship_weights['Ship Cost'], 
                                         model.predict(ship_weights[feature_cols]), squared=False))
    else:
        model = None

    shipping_estimates = []
    for _, row in df.iterrows():
        cssku = row['MasterSKU']
        order_history = warehouse_shipping_data[(warehouse_shipping_data['CS-SKU']==cssku)]
        
        
        # First, check most recent price files for shipping info.
        # This allows for a gradual transition into the new price file scheme.
        if f'{warehouse_key}{cssku}' in price_file_shipping:
            shipping_estimates.append(price_file_shipping[f'{warehouse_key}{cssku}'])
        
        elif len(order_history) > 0:
            # if we have historicals on this one, just take the mean
            shipping_estimates.append(order_history['Ship Cost'].mean())
        elif model:
            # if not, then estimate it using all other skus for this warehouse, with weight/dim
            shipping_estimates.append(model.predict(row[feature_cols].fillna(0).values.reshape(1, -1))[0])
        elif len(warehouse_shipping_data['Ship Cost']) > 0:
            # and if no model available, just use average
            shipping_estimates.append(warehouse_shipping_data['Ship Cost'].mean())
        else:
            shipping_estimates.append(10) # baseline of $10... ??

    df['Shipping'] = shipping_estimates
    return df['Shipping']

# Expects something formatted like a price/weight DF, filtered for a warehouse
# returns the price/weight DF with shipping altered
def calculate_warehouse_shipping(df, warehouse):

    df = df.copy()
    print(warehouse)
    if warehouse=='D': # Dorman
        df.loc[(df['item_cost'] <= 30), ['Shipping', 'ShipMkup']] = 6, DEFAULT_SHIP_MARKUP # flat rate
        df.loc[(df['item_cost'] > 30), ['Shipping', 'ShipMkup']] = 0, 1
        ##df['ShipMkup'] = 1
    elif warehouse=='C': # Brock
        df.loc[(df['item_cost'] <= 50), 'Shipping'] = 16 # estimate / avg.
        df.loc[(df['item_cost'] > 50), 'Shipping'] = 0
        df['ShipMkup'] = 1 / 1.1 # to account for returns not being accepted
    elif warehouse=='P': # Parts Auth
        df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
        '''
        with open(PARTS_AUTH_SHIPPING_MODEL, 'rb') as f:
            m = pickle.load(f)
        df['lwh'] = df['Length'] * df['Width'] * df['Height']
        df['Shipping'] = m.predict(df[['Weight','Length','Width','Height','lwh']].fillna(0))
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
        '''
    elif warehouse=='1': # Jante
        df['Shipping'] = 0
        df['ShipMkup'] = 1
    elif warehouse=='J': # PFG
        PFG_DEFAULT_SHIPPING = 15
        inv = pd.read_csv('inventory/pfg.txt', sep='\t', encoding_errors='ignore', escapechar = '\\', low_memory=False)[['SKU','SHIPPING_COST','HANDLING_COST']]
        parts = inv['SKU'].values
        shippings = []
        for _, row in df.iterrows():
            pn = row['Part Number']
            if pn in parts:
                shippings.append(inv[inv['SKU']==pn].iloc[0][['SHIPPING_COST','HANDLING_COST']].sum())
            else:
                shippings.append(PFG_DEFAULT_SHIPPING)
        df['Shipping'] = shippings
    
        # ADD AN EXTRA $3.50 TO ACCOUNT FOR TAX MESS
        df['Shipping'] = df['Shipping'] + 3.50 
        
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
    elif warehouse=='K': # Keystone
        KEYSTONE_BASE_SHIPPING = 11.0
        KEYSTONE_LTL_SHIPPING = 125.0
        inv = pd.read_csv('inventory/keystone.csv', low_memory=False)
        # Since inventory file open, manage duplicate part # issue in Keystone by matching with UPC
        inv['PartNumber'] = inv['PartNumber'].str.replace('=','').str.replace('"','')
        ##inv['KeystoneShipping'] = (inv['UPS_Ground_Assessorial'] + KEYSTONE_BASE_SHIPPING).fillna(0)
        inv['KeystoneShipping'] = (KEYSTONE_BASE_SHIPPING)
        inv.loc[inv['UPSable']==False, 'KeystoneShipping'] = KEYSTONE_LTL_SHIPPING
        inv = inv.sort_values('KeystoneShipping', ascending=False).drop_duplicates(subset=['VendorCode','PartNumber'])
        df = df.merge(inv[['VendorCode','PartNumber','KeystoneShipping']], 
                      how='left', left_on=['LC','Part Number'], right_on=['VendorCode','PartNumber'])
        df['Shipping'] = df['KeystoneShipping']
        df['ShipMkup'] = 1

    elif warehouse=='6': # Burco Mirrors
        #df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        #df['ShipMkup'] = DEFAULT_SHIP_MARKUP
        df['Shipping'] = 8 # estimate / avg
        df['ShipMkup'] = 1
    elif warehouse=='A': # APW
        df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
    elif warehouse=='2': # OE Wheels
        df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
    elif warehouse=='5': # KSI Trading
        df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
    
    elif warehouse=='7': # NTW
        df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
    elif warehouse=='8': # Race Sport Lighting
        ##df['Shipping'] = df['Weight'].map(lambda w: 15 if (pd.isna(w) or w >= 1) else 6)
        ##df['ShipMkup'] = DEFAULT_SHIP_MARKUP
        df['Shipping'] = 18
        df['ShipMkup'] = 1
    elif warehouse=='9': # Sunbelt APG
        df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
    
    elif warehouse=='N': # NPW
        ##df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        df.loc[df['LC']=='ACD', 'Shipping'] = 35 # set AC Delco skus to $35 shipping
        df['Shipping'] = 12
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
    
    elif warehouse=='O': # Tonsa
        df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP

    elif warehouse=='Y': # MotorState
        df.loc[(df['item_cost'] <= 39), 'Shipping'] = 13
        df.loc[((df['item_cost'] > 39) & (df['item_cost'] <= 99.99)), 'Shipping'] = 11.5
        df.loc[(df['item_cost'] >= 100), 'Shipping'] = 10
        df['ShipMkup'] = 1
    else:
        pass
    df['ShipMkup'] = df['ShipMkup'].fillna(1)
    return df[['Shipping','ShipMkup']]

dfs = []
for warehouse in pw['WD'].unique().tolist():
    wdf = pw[pw['WD']==warehouse].copy()
    wdf.loc[:, ['Shipping','ShipMkup']] = calculate_warehouse_shipping(wdf.loc[wdf['WD']==warehouse, :], 
                                                                           warehouse).values
    print('Proportion of parts missing shipping:', wdf['Shipping'].isna().mean())
    dfs.append(wdf)
pw = pd.concat(dfs, ignore_index=True)
pw['ShipMkup'] = pw['ShipMkup'].round(3)

#### Set inventory constraints

In [ ]:
pw.loc[:, ['MinQty','MaxQty']] = 2, 12 ## Changed Min to two from 3

#### Set price file defaults.

In [ ]:
pw['ListMkup'] = .65
pw['SourceQty'] = None
pw['Source'] = None
pw['BundleSK'] = None
pw['Carrier'] = 'FedEx'
pw['Service'] = 'GroundHD'

In [ ]:
pw.loc[(pw['CS-SKU-NP'].str[0]=='D'), ['OP-Lowest(Y)','VND-Lowest(Y)'] ] = "N", "N"

## Quick fix for shipping

Where shipping is current zero for Tonsa, set it to 20. And 15 for Sunbelt.

In [ ]:
pw.loc[((pw['Shipping'] == 0) & (pw['CS-SKU-NP'].str[0] == 'O')), 'Shipping'] = 20
pw.loc[((pw['Shipping'] == 0) & (pw['CS-SKU-NP'].str[0] == '9')), 'Shipping'] = 15

1.5x Tonsa, Sunbelt, and Parts Auth shipping.

In [ ]:
pw.loc[(pw['CS-SKU-NP'].str[0]=='P'), 'Shipping'] *= 1.5

Double Eagle Eye Shipping.

In [ ]:
pw.loc[(pw['CS-SKU-NP'].str[:4]=='P754'), 'Shipping'] *= 2.0

Bumper is expensive to ship.

In [ ]:
pw.loc[pw['CS-SKU-NP']=='429|6448-0006', 'Shipping'] = 30

Another expensive shipping update from order: PSA669874628

In [ ]:
pw.loc[pw['CS-SKU-NP']=='551|S6585B', 'Shipping'] = 46

Manual SKU Changes:

In [ ]:
pw.loc[pw['CS-SKU-NP']=='P576|3292', 'Shipping'] = 90

In [ ]:
pw.loc[pw['CS-SKU-NP']=='P550|290073', 'Shipping'] = 100

In [ ]:
pw.loc[pw['CS-SKU-NP']=='P308|55621', 'Shipping'] = 161

In [ ]:
pw.loc[pw['CS-SKU-NP']=='P557|277504', 'Shipping'] = 35

In [ ]:
pw.loc[pw['CS-SKU-NP']=='P643|ESK5752', 'Shipping'] = 34

In [ ]:
pw.loc[pw['CS-SKU-NP']=='P551|40722A', 'Shipping'] = 7

In [ ]:
pw.loc[pw['CS-SKU-NP']=='N643|AR8265XPR', 'PackQty'] = 1

In [ ]:
pw.loc[pw['CS-SKU-NP']=='P123|33660', 'PackQty'] = 1

PA Shipping Costs from Umer analysis:

In [ ]:
pa_shipping_data = pd.read_csv('PA Shipping Costs.csv', low_memory=False)
pa_shipping_data['WD'] = 'P'

pa_shipping_data = pa_shipping_data.sort_values(by='Row Labels', ascending=False)
pa_shipping_data.drop_duplicates(subset='Final Shipping Cost', keep="first")

pw = pw.merge(pa_shipping_data, how='left', left_on=['MasterSKU', 'WD'], right_on=['Row Labels', 'WD'])
pw.loc[pw['Final Shipping Cost'] > 0, 'Shipping'] = pw['Final Shipping Cost']

In [ ]:
new_shipping = pd.read_csv('newshipcosts.csv', low_memory=False)

pw = pw.merge(new_shipping, how='left', left_on=['MasterSKU'], right_on=['CS_SKU'])
pw.loc[pw['NewShipCost'] > 0, 'Shipping'] = pw['NewShipCost']

Set MinQty really high for First Stop Brakes (Dorman line) to avoid actually selling any.

In [ ]:
first_stop_brakes = pd.read_excel('8-2 Change 56 Brake Dropship and Stocking.xlsx', 
                                  skiprows=2, sheet_name='Dropship Price').rename(columns={'MATERIAL':'pn'})['pn']
pw.loc[(pw['WD']=='D') & pw['Part Number'].isin(first_stop_brakes), ['MinQty','MaxQty']] = 100, 100

Handle RSL skus for MAP > Calculated Cost

In [ ]:
rsl_inventory = pd.read_csv('inventory/rsl.csv', low_memory=False)
rsl_inventory = rsl_inventory.to_dict('records')
rsl_skus_map_price = []

for row in rsl_inventory:
    if ("8329|" + row['SKU']) in pw['CS-SKU-NP'].values:
        matched_rsl = pw.loc[pw['CS-SKU-NP'] == "8329|" + row['SKU']]
        pw_item_cost = matched_rsl['item_cost'].values[0]
        pw_shipping = matched_rsl['Shipping'].values[0]
        markup = pw_item_cost / ( (1 + 0.05) * (pw_item_cost + pw_shipping) / (1 - 0.15) - pw_shipping)
        if (pw_item_cost / markup) + (pw_shipping / matched_rsl['ShipMkup'].values[0]) < row['MAP']:
            pw.loc[pw['CS-SKU-NP'] == "8329|" + row['SKU'], ['MinPrice', 'item_cost', 'Shipping', 'ShipMkup']] = row['MAP'], row['MAP'], 0, 0
            rsl_skus_map_price.append("8329|" + row['SKU'])

Motorstate Formatting:

In [32]:
motorstate = pd.read_csv('inventory/motorstate.csv', low_memory=False)
#motorstate['PartNumber'] = motorstate.PartNumber.str[3:]
motorstate['PartNumber'] = 'Y' + motorstate['PartNumber']
motorstate
#pw = pw.merge(motorstate, how='left', left_on=['Part Number', 'WD'], right_on=['PartNumber', 'WD'])


pw['tkey'] = pw['WD'] + pw['LC'] + pw['Part Number']
pw_cols = pw.columns
pw = pw.merge(motorstate, how='left', left_on=['tkey'], right_on=['PartNumber'])
pw.drop(columns='tkey', inplace=True)

In [ ]:
pw.drop(pw.loc[pw['AirRestricted'] == 'YES'].index, inplace=True)
pw.drop(pw.loc[pw['TruckFrtOnly'] == 'YES'].index, inplace=True)

In [ ]:
pw.drop(pw.loc[(pw['WD'] == 'Y') & (pw['MasterLC'] != 261)].index, inplace=True)

NPW Min Order Qty

OE wheels price MAP fix 11/09

In [ ]:
oe_wheel_update = pd.read_excel('OE Wheel Shipping.xlsx', sheet_name='Sheet1')
oe_wheel_update = oe_wheel_update.to_dict('records')

for row in oe_wheel_update:
    if pw['CS-SKU-NP'].isin(["2387|" + str(row['UPC'])]).any:
        pw.loc[pw['CS-SKU-NP'] == "2387|" + str(row['UPC']), 'Shipping'] = row['Shipping (Est)']

Fix for Batteries shipping costs

In [ ]:
batteries = pd.read_excel('Input File PA Battery.xlsx', sheet_name='Sheet1')
batteries['Part'] = batteries['Part'].str.replace('-','')
batteries = batteries.to_dict('records')

for row in batteries:
    print(row['Part'])
    if pd.isna(row['Part']) == False and pw['CS-SKU-NP'].str[5:].isin([row['Part']]).any:
        pw.loc[pw['CS-SKU-NP'].str.contains(row['Part']), 'Shipping'] = 50

OE Wheels 4PLAY 20% Discount

In [ ]:
'''
oe_wheels_discount = pd.read_csv('4PLAY Wheels - Sheet1.csv', low_memory=False)
oe_wheels_discount = oe_wheels_discount.to_dict('records')

for row in oe_wheels_discount:
    if pw['CS-SKU'].isin(["2" + row['4Play wheels']]).any:
        wheel_result = pw.loc[pw['CS-SKU'] == "2" + row['4Play wheels']]
        print(wheel_result['MinPrice'].values[0])
        twenty_percent = (wheel_result['MinPrice'].values[0] / 10) * 2
        print(twenty_percent)
        pw.loc[pw['CS-SKU'] == "2" + row['4Play wheels'], ['MinPrice', 'MAP']] = wheel_result['MinPrice'].values[0] - twenty_percent, wheel_result['MinPrice'].values[0] - twenty_percent
'''        

Use PA Inventory file for PackQtys

In [ ]:
pa_inv = pd.read_csv('inventory/pa.csv', low_memory=False, encoding= 'unicode_escape')
pa_inv['Part'] = pa_inv['Part'].replace('-', '', regex=True)


pw = pw.merge(pa_inv, how='left', left_on=['Part Number', 'LC'], right_on=['Part', 'Line'])
pw.loc[pw['Packs'] > 1, 'PackQty'] = pw['Packs']

Force shipping cost and ship markup for RSL Skus

In [ ]:
pw.loc[(pw['WD'] == '8') & (pw['Shipping'] == 0), ['Shipping', 'ShipMkup']] = 18, 1

#### Calculate markups and format/write price files, by channel.

In [ ]:
noncats = pd.read_csv('non-cat-skus.csv', skiprows=1, low_memory=False)[['CS Linecode','Part Number']]
noncats['cssku'] = noncats['CS Linecode'] + '|' + noncats['Part Number']
noncat_skus = noncats['cssku'].dropna()

In [ ]:
for channel in CHANNELS:
    pf = pw.copy()
    
    # Filter price file if APF or Mecka
    if channel['name'] == 'AP Fusion':
        apf_csskus = pd.read_csv('apf-skus.csv')['cssku']
        apf_csskus = apf_csskus.drop_duplicates()
        pf = pf[pf['CS-SKU-NP'].str[1:].isin(apf_csskus)]
    if channel['name'] == 'Mecka':
        pf = pf[(pf['WD']=='D') & (pf['item_cost'] >= 30)]
    # FIlter BS Walmart 
    if channel['name'] == 'BS Walmart':
        print(channel['name'])
        pf.drop(pf.loc[pf['WD'] == 'Y'].index, inplace=True)
        ##pf.drop(pf.loc[pf['WD'] == 'P'].index, inplace=True)
        pf.drop(pf.loc[pf['WD'] == 'J'].index, inplace=True)
        pf.drop(pf.loc[pf['WD'] == 'C'].index, inplace=True)
        pf.drop(pf.loc[pf['WD'] == '8'].index, inplace=True)

    # Calculate markups
    target_profit = channel['target_profit'] if 'target_profit' in channel else DEFAULT_TARGET_PROFIT
    channel_fees = channel['channel_transaction_fee'] + CS_TRANSACTION_FEE
    markups = []
    for _, row in pf.iterrows():
        # handle brock specially
        if row['CS-SKU-NP'][0] == 'C':
            our_cost = row['item_cost']
            if row['item_cost'] <= 50: # + $6 for any item costing < $40
                our_cost += 4
            our_cost *= 1.1 # 10% marup for all items, as buffer against no-return policy
            # calculate markup using same formula but with our updated cost value
            our_markup = our_cost / ( (1 + target_profit) * (our_cost + row['Shipping']) / (1 - channel_fees) - row['Shipping'])
            # then adjust to find what the equivalent markup would be (to get to same final price) with the original cost value
            markup = row['item_cost'] * our_markup / our_cost
        
        # handle Jante, certain skus specially
        elif ( channel['name'] == 'PS Ebay' and row['CS-SKU-NP'][:4] == '1367' ):
            ##markup = row['item_cost'] / ( (1 + target_profit) * (row['item_cost'] + row['Shipping']) / (1 - (channel_fees - .02)) - row['Shipping'])
            markup = row['item_cost'] / ( (1 + target_profit) * (row['item_cost'] + row['Shipping']) / (1 - channel_fees) - row['Shipping'])
        # Added 1% more to channel fees
        elif ( channel['name'] == 'PS Ebay' and row['CS-SKU-NP'][0] == '1' ):
            markup = row['item_cost'] / ( (1 + target_profit) * (row['item_cost'] + row['Shipping']) / (1 - (channel_fees + 0.01)) - row['Shipping'])
        # Dorman increase profit
        elif row['CS-SKU-NP'][0] == 'D':
            markup = row['item_cost'] / ( (1 + 0.06) * (row['item_cost'] + row['Shipping']) / (1 - channel_fees) - row['Shipping'])  
            
        # base case, all other skus
        else:
            markup = row['item_cost'] / ( (1 + target_profit) * (row['item_cost'] + row['Shipping']) / (1 - channel_fees) - row['Shipping'])

        markups.append(markup)
    pf['Markup'] = markups
    
    # Format price file.
    if channel['name'] in CatSKU_CHANNELS:
        pf['CatSKU'] = (~pf['CS-SKU-NP'].str[1:].isin(noncat_skus)).map(lambda x: 'Y' if x else 'N')
        pf.loc[pf['CatSKU']=='Y', 'CS-SKU-NP'] = pf.loc[pf['CatSKU']=='Y', 'CS-SKU-NP-CatSKU']
    else:
        pf['CatSKU'] = 'N'
    
    # Clean up
    pf.loc[pf['MinPrice'] < 1,'MinPrice'] = 1
    pf.loc[pf['Markup'] < .1,'Markup'] = .1
    pf.loc[pf['Markup'] > 1,'Markup'] = 1
    pf['Markup'] = pf['Markup'].round(3)
    pf['Shipping'] = pf['Shipping'].round(2)
    pf.loc[pf['Shipping'].lt(0), 'Shipping'] = 0
    pf['total_cost'] = pf['item_cost'] + pf['Shipping'] # hoping this will fix most examples of Dorman going thru PA
    
    # give dorman direct preference over dorman skus from other warehouses
    
    #Dorman Pref - Original
    #dorman_prefs = pw[(pw['WD']=='D') & (pw['Qty'] > 0)]['MasterSKU'].copy().tolist()
    #pw = pw[~((pw['WD'] != 'D') & (pw['MasterSKU'].isin(dorman_prefs)))] 
    
    #Dorman Pref - Corrected
    dorman_prefs = pf[(pf['WD']=='D') & (pf['Qty'] > 0)]['MasterSKU'].copy().tolist()
    pf = pf[~((pf['WD'] != 'D') & (pf['MasterSKU'].isin(dorman_prefs)))]   #Corrected - The change should be wpplied on current price file, not pw
    
    pf = pf.sort_values(['MasterSKU','Qty','total_cost'], ascending=[True,False,True])\
           .drop_duplicates(subset=['MasterSKU'], keep='first')
    
    pf = pf[PRICE_FILE_COLUMNS]
    
    # Write price file.
    pf.to_csv(f"{PRICE_FILE_LOCATION}/{channel['name']}.csv", index=False)
    print(channel)